# Sudoku Generator

Importing the libraries:

In [1]:
import numpy as np

## Auxiliary functions

Checks if the grid satisfies the single element constraint and if the target state has been reached:

In [2]:
N = 9  # number of grid rows and columns
STEP = 3  # defines the side of the quadrant

def sudoku_constraint(grid_slice):
    return np.array_equal(np.unique(grid_slice), np.arange(1, N+1))

def check_rows(grid):
    for row in grid:
        if not sudoku_constraint(row):
            return False
    
    return True

def check_columns(grid):
    for column in grid.T:
        if not sudoku_constraint(column):
            return False
    
    return True

def check_quadrants(grid):
    for i in range(0, grid.shape[0], STEP):
        for j in range(0, grid.shape[1], STEP):
            if not sudoku_constraint(np.unique(grid[i:i+STEP, j:j+STEP])):
                return False
    
    return True

def objective_grid(grid):
    return check_rows(grid) and check_columns(grid) and check_quadrants(grid)

Checking available actions for a given grid:

In [3]:
def number_constraint(grid, y, x, num):
    yy = y // STEP * 3
    xx = x // STEP * 3
    
    return num not in np.concatenate((grid[y], grid[:, x], grid[yy : yy+STEP, xx : xx+STEP].flatten()))

def flatten_position(pos):
    return pos // N, pos % N

def available_pos(grid):
    return [(y, x) for y, x in zip(*np.where(grid == 0))]

def available_nums(grid, y, x):
    return [num for num in range(1, N+1) if not grid[y, x] and number_constraint(grid, y, x, num)] 

def available_actions(grid):
    return [((y, x), num) for y, x in available_pos(grid) for num in available_nums(grid, y, x)]

## Generator

Using the backtracking search strategy:

In [4]:
def generator():
    grid = np.zeros((N, N))
    
    moves = [None] * 81
    size = 0
    while True:
        if objective_grid(grid):
            for i in range(40):
                pos = flatten_position(np.random.randint(81))
                grid[pos] = 0
                
            return grid
        
        # Getting the next available position and fill options
        pos = flatten_position(size)
        nums = available_nums(grid, *pos)
        
        if nums:    
            num = nums.pop(np.random.randint(len(nums)))
        
            grid[pos] = num
            moves[size] = nums
            size += 1
            
            continue
                
        grid[pos] = 0
        size -= 1

        while not moves[size]:
            pos = flatten_position(size)
            
            grid[pos] = 0
            size -= 1

        pos = flatten_position(size)
        num = moves[size].pop(np.random.randint(len(moves[size])))

        grid[pos] = num
        size += 1

Testing the sudoku generator:

In [5]:
grid = generator()

grid

array([[8., 3., 0., 0., 6., 1., 7., 4., 9.],
       [0., 6., 1., 2., 7., 0., 3., 5., 0.],
       [4., 5., 7., 3., 9., 8., 2., 0., 0.],
       [0., 0., 9., 0., 5., 0., 0., 0., 6.],
       [0., 8., 0., 0., 0., 0., 1., 9., 3.],
       [6., 0., 3., 8., 0., 0., 4., 0., 0.],
       [7., 4., 0., 9., 0., 0., 6., 0., 0.],
       [2., 1., 0., 0., 8., 6., 9., 3., 4.],
       [3., 0., 0., 1., 4., 2., 0., 0., 7.]])